# HW2- Steam Reviews


# Loading the Dataset

In [1]:
'''
from google.colab import drive
drive.mount('/content/drive/')
''';


 This is a dataset of around 21 million user reviews of around 300 different games on Steam: sto importando solo poche righe per lavorarci

In [2]:
def read_csv_with_time(path, time_fields, n_rows, usecols=None):
    '''
      This function reads a csv and returns a dataframe considering only the first n_rows rows
      and transforming the indicated time_fields from timestamp(seconds) in datetime objects

      Arguments
      _________
        path: str
          The path where the file is located
        time_fields: List[str]
          A list of the fields to be converted in datetime
        n_rows: int
          The number of rows to be considered
        usecols: List[str]
          The list of the columns to be loaded      
      Returns
      _______
        a pandas dataframe containing the processed file
    '''
    
    return pd.read_csv(path, header='infer', nrows=n_rows, 
        parse_dates= [tf for tf in time_fields], date_parser=lambda x: pd.to_datetime(x, unit='s'), usecols=usecols)

def hour_in_range(str_hour, range_hour):
    '''
      Given a string defining an hour and a range of hour as a tuple of that type of string,
      the function assert when the given hour is in the range

      Arguments
      _________
        str_hour: str
          in the format HH:MM:SS
        range_hour: Tuple(str)
          a tuple of string in the form (HH:MM:SS, HH:MM:SS) where the first hour is lower than the second
    '''
    
    min_hour, max_hour = range_hour
    assert (hour_comparator(min_hour, max_hour) == -1), "A range is valid only if the first element is lower than the second"
    return (hour_comparator(str_hour, min_hour) * hour_comparator(max_hour, str_hour)) >=0

def hour_comparator(str_h1, str_h2):
    '''
      Compares two string in the format HH:MM:SS and returns an integer value accordingly with their comparison
      
      Arguments
      __________
        str_h1: str
          in the format HH:MM:SS
        str_h2: str
          in the format HH:MM:SS
      
      Returns
      _______
        An integer representing the comparison between the given strings:
          -1  if the first is less then the second
           0  if the dates are the same
           1  if the first is greater than the second
    '''
    hh1, mm1, ss1 = map(int,str_h1.split(':'))
    hh2, mm2, ss2 = map(int, str_h2.split(':'))
    deltas = [hh1-hh2, mm1-mm2, ss1-ss2]
    for d in deltas:
        if d>0:
            return 1
        elif d < 0:
            return -1
    return 0

def get_range_index(str_hour, ranges):
    '''
      Given a string hour and a list of hour ranges, the function returns the index of the range
      to wich the string hour belongs

      Arguments
      _________
        str_hour: str
          in the format HH:MM:SS
        ranges: List[Tuple(str)]
          List containing hour ranges, so list o tuples of string in the form
          (HH:MM:SS, HH:MM:SS) where the first hour is lower than the second
      
      Return
      ______
        an integer indicating the index of the range where the hour is
    '''
    for i in range(len(ranges)):
        if hour_in_range(str_hour, ranges[i]):
            return i
    return -1

def transform_in_hour_ranges(df, column, ranges):
    '''
      Given a dataframe convert the given column of datetime to the index of the range in the given list
      where the value belongs

      Arguments
      _________
        df: pd.DataFrame
        column: str
          the name of a Datetime column of the df
        ranges: List[Tuple(str)]
          List containing hour ranges, so list o tuples of string in the form
          (HH:MM:SS, HH:MM:SS) where the first hour is lower than the second

      Return
      ______
        the df with the given column modified

    '''
    df[column]=df[column].apply(lambda x: get_range_index(x.strftime('%H:%M:%S'), ranges))
    return df


# REALLY USED:

#read_csv_with_time

def get_integer_ranges(ranges):
    return [(tuple(int(data) for data in x[0].split(':')), tuple(int(data) for data in x[1].split(':'))) for x in ranges]

def get_integer_range_index(tuple_hour, integer_ranges):
    
    for i in range(len(integer_ranges)):
        min_r, max_r = integer_ranges[i]
        if tuple_hour >= min_r and tuple_hour <= max_r:
            return i
    return -1

In [3]:
#fname = '/content/drive/MyDrive/HW2-ADM/steam_reviews.csv'
fname = '/content/drive/MyDrive/HW2-ADM/steam_reviews.csv'
ts_created = 'timestamp_created'

def_ranges = [('06:00:00', '10:59:59'),
('11:00:00', '13:59:59'),
('14:00:00', '16:59:59'),
('17:00:00', '19:59:59'),
('20:00:00', '23:59:59'),
('00:00:00', '02:59:59'),
('03:00:00', '05:59:59')]

### Usefull libraries

In [4]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt

  


In [5]:
#df= pd.read_csv('/content/drive/MyDrive/HW2-ADM/steam_reviews.csv',nrows=10000000)


In [ ]:
df=pd.read_csv('steam_reviews.csv',nrows=21000000)

# [RQ1]: Exploratory Data Analysis

In [ ]:
df.head(3)

In [ ]:
df.info()

We can see that there are 23 columns, 8 integer,5 decimal, 4 boolean and 6 of other type.
The first one 'Unnamed' can be consider just like an index.


In [ ]:
df.isnull().sum()

It appears that the variables with missing values ​​are: 'review' (which contains the text itself) , 'author.playtime_at_review' (Author playtime of reviewed app at time of review) ,   .

In [ ]:
df.describe()

In [ ]:
import seaborn as sns

In [ ]:
#seaborn.heatmap(df)
df_kor = df.corr()
plt.figure(figsize=(10,10))
sns.heatmap(df_kor, vmin=-1, vmax=1, cmap="viridis", annot=True, linewidth=0.1)



Now let's focus on the variables that we consider more interesting. We start with app_name.

In [ ]:
#App_name
x=df["app_name"].value_counts(normalize=True) #in percentuale le diverse app recensite
print(x)
names=df["app_name"].unique() #nomi app
print(names)
label=["Tom Clancy's Rainbow Six Siege","Garry's Mod","Rust" ]

#x_1=x[:3:]
#print(x_1)

#Grafico a torta
x.plot.pie() #labels=label)
plt.show()

#Farei un barpot con quelle "vere"
plt.bar(names,x)
plt.xticks(rotation='vertical')

In total there are 315 different apps, the most common are: PLAYERUNKNOWN'S BATTLEGROUNDS (8%),Grand Theft Auto V (5%) and.. (bo va visto sui dati tot).


Now we consider the language. The most common are...

In [ ]:
print(df["language"].value_counts(normalize=True))
data=[0.46,0.14,0.11]
labels = ['english', 'chinese', 'russian']
plt.xticks([1,2,3], labels)
plt.xlabel('Language')
plt.ylabel('Percentage')
plt.title("Language's barplot")
plt.bar([1,2,3], data)
plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)
plt.ylim(0.0,1)
plt.show()



Recommended and steam_purchase 
We observe that 92% of review authors recommend the app and 78% of them purchased the app on Steam. But only 72% of the authors give a positive opinion and then actually buy the app.

In [ ]:
print(df["recommended"].value_counts(normalize=True))

print(df["steam_purchase"].value_counts(normalize=True))

len(df[  (df.recommended == True) &(df.steam_purchase == True)  ])/ 10000000


author.num_reviews :Number of lifetime app reviews by author
Con i dati veri si può fare un commento.


In [ ]:
print(df["author.num_reviews"].describe())


# RQ2: Let's explore the dataset by finding simple insights into the reviews.

### Plot the number of reviews for each application in descending order.

In [ ]:
new = df[['app_name']]
a=new["app_name"].value_counts()
app_sort=a.sort_values(ascending=False) #lo salvo perchè serve dopo
app_sort

### What applications have the best Weighted Vote Score?

In [ ]:
#In this way  the apps are sorted by  the score and then we can consider the top 5/10 (is up to us).
new_1 = df[['app_name','weighted_vote_score'  ]]
new_1.sort_values(by='weighted_vote_score',ascending=False)
#In this way I compute the average score for each app
new_1.groupby('app_name').agg({"weighted_vote_score":"mean"})
#LO lascerei svolto in entrambi i modi, come dicono su slack. Non gredo che agg (aggregate) sia necessario ma non sapevo  farlo senza. Non faccio i commenti perchè tanto i risultati finali saranno diversi.

### Which applications have the most and the least recommendations?

In [ ]:

y=df.groupby('app_name').agg({"recommended":"sum"})
#print(y)

y=y.sort_values(by="recommended",ascending=False)
#print('the most recommended application is ...... with a total number of recommendations of .....', )
y[0:5] #most
#print('the worst recommended application is ..... with a total number of recommendations of ....', )
#y[-1:] #least


### How many of these applications were purchased, and how many were given for free?

In [ ]:

print(df["steam_purchase"].value_counts(normalize=True))
print(df["received_for_free"].value_counts(normalize=True))
#Steam purchased: True se l'autore ha comprato l'app su Steam
#Received for free: True se ha ricevuto l'app gratis.
#non so bene quale vuole, per me il secondo ma boo

# RQ3: Now it's important to understand the preferred time to do reviews.




### Request 1


I'll use the already loaded-in-memory dataset (in the variable df) in order to map every timestamp_created entry from the Datetime format, to a tuple made by the integer value of hour and minutes, once it's done i'll use the obtained column to group by the mapped rows counting the occourrence of every distinct row, i've subsequently used that pd.Series in order to extract the couple (hour, minutes) that maximizes the number of rows, so the number of reviews made in that hour range

The results are printed with a formatted string

In [ ]:
from datetime import datetime
df = read_csv_with_time(fname, ['timestamp_created'], n_rows=10e6, usecols=['timestamp_created'])

# Converting the timestamps in tuples made by hours and minutes
timed = df.timestamp_created.apply(lambda x: (x.hour, x.minute))
# grouping by values (the tuples of above) and counting the occourrences
timed = timed.groupby(timed).count() 
amax = timed.argmax() # Getting the index of the maximum value

# Retrievieng the answers and formatting them...
hh, mm = timed.index[amax]
n_rev = timed.values[amax]
hh = "{:0>2d}".format(hh)
mm = "{:0>2d}".format(mm)

print(f"The most common hour in wich are published the greater part of the reviews is at {hh}:{mm} with {n_rev} reviews published")

### Request 2

The function required is the following and returns a pd.Series where the index are the index of the corresponding hour range and the values are the number of reviews made in that range

In [ ]:
def new_bis(ranges, ds=fname, n_rows=1e6):
    df = read_csv_with_time(ds, ['timestamp_created'], n_rows, usecols=['timestamp_created'])
    ranges_int = get_integer_ranges(ranges)
    ranged = df.timestamp_created.apply(lambda x: get_integer_range_index((x.hour, x.minute, x.second), ranges_int))
    ranged = ranged.groupby(ranged).count()
    return ranged

### Request 3

I've used the function above in order to plot by an horizontal bar chart, the number of reviews made for every hour range of the default ranges shown in the table above

In [ ]:
ret = new_bis(def_ranges, n_rows=30e6)
plt.figure(figsize=(20,10))
ax = ret.plot(kind="barh", color='green')
ax.set_title("Reviews per hour ranges")
ax.set_ylabel("hour ranges")
ax.set_xlabel("Num of reviews")
ax.set_yticklabels(def_ranges)
ax.grid(b=True, color='grey', linestyle='-.', linewidth=0.5, alpha=0.2)

In [ ]:
#previous code
import pandas as pd
'''


def read_csv_with_time(path, time_fields, n_rows):
    return pd.read_csv(path, header='infer', nrows=n_rows, 
        parse_dates= [tf for tf in time_fields], date_parser=lambda x: pd.to_datetime(x, unit='s'))

def hour_in_range(str_hour, range_hour):
    min_hour, max_hour = range_hour
    return (hour_comparator(str_hour, min_hour) * hour_comparator(max_hour, str_hour)) >=0

def hour_comparator(str_h1, str_h2):
    
        #Compares two string in the format HH:MM:SS and returns:
        #    -1 if the first is less then the second
        #     0  if the dates are the same
        #    1 if the first is greater than the second
    
    hh1, mm1, ss1 = map(int,str_h1.split(':'))
    hh2, mm2, ss2 = map(int, str_h2.split(':'))
    deltas = [hh1-hh2, mm1-mm2, ss1-ss2]
    for d in deltas:
        if d>0:
            return 1
        elif d < 0:
            return -1
    return 0

def get_range_index(str_hour, ranges):
    for i in range(len(ranges)):
        if hour_in_range(str_hour, ranges[i]):
            return i
    return -1

def transform_in_hour_ranges(df, column, ranges):
    df[column]=df[column].apply(lambda x: get_range_index(x.strftime('%H:%M:%S'), ranges))
    return df

fname = '/content/drive/MyDrive/HW2-ADM/steam_reviews.csv'
ts_created = 'timestamp_created'

def_ranges = [('06:00:00', '10:59:59'),
('11:00:00', '13:59:59'),
('14:00:00', '16:59:59'),
('17:00:00', '19:59:59'),
('20:00:00', '23:59:59'),
('00:00:00', '02:59:59'),
('03:00:00', '05:59:59')]

df = transform_in_hour_ranges(read_csv_with_time(fname, [ts_created], 1000000), ts_created, def_ranges)

import matplotlib.pyplot as plt
import numpy as np
labels = ['pippo' for df in def_ranges]
plt.xticks(list(range(len(labels))), labels)
#plt.setxticks
df.groupby(df.timestamp_created).review_id.count().plot.bar()
''';

# RQ4 As Steam is a worldwide platform, the reviews can be done in many languages. Let's extract some information about it.

### What are the top 3 languages used to review applications?

In [ ]:
lingue=df["language"].value_counts(normalize=True)
top=lingue[:3]
top_list=list(top.index)
print(*(x for x in top_list), sep='\n')
#top five

### Create a function that receives as parameters both the name of a data set and a list of languages’ names and returns a data frame filtered only with the reviews written in the provided languages.

In [ ]:

def filter_language(dataset, languages,col_name):
  return dataset.loc[dataset[col_name].isin(languages)]


### Use the function created in the previous literal to find what percentage of these reviews (associated with the the top 3 languages) were voted as funny?

In [ ]:
for i in top_list:
     print(i+"   "+str(round(sum((df['votes_funny']==0) & (df['language']==i))/sum(df['language']==i),2))+"%")


### Use the function created in the literal “a” to find what percentage of these reviews (associated with the top 3 languages) were voted as helpful?

In [ ]:
df_1= filter_language(df,top_list,'language') #dati filtrati
for i in top_list:
  print(i+"   "+str(round(sum((df['votes_helpful']!=0) & (df['language']==i))/sum(df['language']==i),2))+"%")

# RQ5 The reviews' authors are users from the game that provide their opinion on it. Now you can check how often they make reviews.

### Plot the top 10 most popular reviewers and the number of reviews.

In [ ]:
#df= pd.read_csv('/content/drive/MyDrive/HW2-ADM/steam_reviews.csv',nrows=10000000)
df_auth = df.groupby('author.steamid').review_id.count().sort_values(ascending=False)
df_auth[:10].plot.bar()

### What applications did the most popular author review? 

In [ ]:
from_bigger_reviewer = df[df['author.steamid'] == df_auth[:1].index[0]]


In [ ]:
from_bigger_reviewer['app_name']

### How many applications did he purchase, and how many did he get as free? Provide the number (count) and the percentage.

In [ ]:
free = from_bigger_reviewer[from_bigger_reviewer.received_for_free]
n_free = len(free)
purchased = from_bigger_reviewer[from_bigger_reviewer.steam_purchase]
n_purch = len(purchased)
tot = len(from_bigger_reviewer)
print(f"He got for free {n_free} applications ({n_free/tot*100}%) and purchased {n_purch} ({n_purch/tot*100}%) on total of {tot}")
print('   ')
print(f"The author's recommended {len(free['recommended'])} and doesn't recommended {n_free-len(free['recommended'])} application from the ones received for free")


### How many of the applications he purchased reviewed positively, and how many negatively? How about the applications he received for free?

In [ ]:
print('   ')
print(f" recommended {len(purchased['recommended'])} and doesn't recommended {n_purch-len(purchased['recommended'])} application from the ones purchased")

# [RQ6] It's time to get information from the updates that a user does to his reviews.

### What is the average time (days and minutes) a user lets pass before he updates a review?

In [ ]:
print(f"The average time that pass between the creation and the update of the reviews is: {(df.timestamp_updated - df.timestamp_created).mean(numeric_only=False)}")


### Plot the top 3 authors that usually update their reviews.

In [ ]:
df['updated'] = df.timestamp_created == df.timestamp_updated
grouped = df.groupby(df['author.steamid']).updated.sum().sort_values(ascending=False)
auth = grouped[:3].index
print("The authors that has updated their reviews more often are, in order:")
for a in auth:
  print(f"\t*{a} with {grouped[a]} updated reviews")

# RQ7 Of course, calculating probabilities is a job that any Data Scientist must know. Let's compute Some interesting figures.

### What’s the probability that a review has a Weighted Vote Score equal to or bigger than 0.5?

In [ ]:
#Prob = fav. cases / possible cases
p1=sum(df['weighted_vote_score']> 0.5)/df.shape[0]


### What’s the probability that a review has at least one vote as funny given that the Weighted Vote Score is bigger than 0.5?

In [ ]:
#prob condizionata

intersezione=sum((df['weighted_vote_score']> 0.5)& (df['votes_funny']!=0))/df.shape[0]
p2=intersezione/p1
p2

### Is the probability that “a review has at least one vote as funny” independent of the “probability that a review has a Weighted Vote Score equal or bigger than 0.5”?

In [ ]:
pa= sum(df['votes_funny']!=0)/df.shape[0]
pa*p1==intersezione
#False, quindi Non sono indipendenti

# RQ8 Every decision you take in a data-based environment should be reinforced with charts, statistical tests and analysis methods to check if a hypothesis is correct or not.

### Is there a significant difference in the Weighted Vote Score of reviews made in Chinese vs the ones made in Russian? Use an appropriate statistical test or technique and support your choicv1

In [ ]:
'''

v1=df.loc[df['language'] == 'russian', 'weighted_vote_score']
v2=df.loc[df['language'] == 'chinese', 'weighted_vote_score']

#!pip3 install  researchpy
#import scipy.stats as stats

#stats.ttest_ind(df['weighted_vote_score'][df['language'] == 'russian'],
                #df['weighted_vote_score'][df['language'] == 'chinese'])
stats.ttest_ind(v1,v2)
''';


In [ ]:
#Is there a significant difference in the Weighted Vote Score of reviews made in Chinese vs the ones made in Russian? Use an appropriate statistical test or technique and support your choicv1
s1=df['weighted_vote_score'][df['language'] == 'russian']
s2=df['weighted_vote_score'][df['language'] == 'schinese']
s1.describe()
s2.describe()
#sono abbastanza diverse!

In [ ]:
#grafici per far vedere che non vale la normalità se teniamo gli zeri.
import seaborn as sns 

z1= s1[s1!=0]
z2= s2[s2!=0]
z2
sns.displot(z1)

sns.displot(z2)

In [ ]:
#Prendiamo gli zeri: la parte discreta
zero_1= s1[s1==0]
zero_2=s2[s2==0]

#booo

In [ ]:
from scipy import stats

In [ ]:
#this is a two-sided test for the null hypothesis that 2 independent samples have identical average (expected) values. 
#Test per la aprte continua
stats.ttest_ind(z1,z2)
#There is a statistically significant difference in the average score between english and chinese, t= -69, p= 0.00.
 #If the p-value is smaller than our threshold, then we have evidence against the null hypothesis of equal population means.
 #The t-test quantifies the difference between the arithmetic means of the two samples

The indepentent T-test is a parametric test used to test for a statistically significant difference in the means between 2 groups. As with all parametric tests, there are certain conditions that need to be met in order for the test results to be considered reliable. 
Hp:
The two samples are independent: accettabile per costruzione (?)
One of the assumptions is that the sampling distribution is normally distributed. N is very big so it's ok.
A way to test the assumption is through a visual check (va fatto non ci riuscivo..)

In [ ]:
import researchpy as rp
summary, results = rp.ttest(group1= z1, group1_name= "Russian", group2= z2, group2_name= "Chinese")
print(summary)
#farò un commento
print(results)

### Can you find any significant relationship between the time that a user lets pass before he updates the review and the Weighted Vote Score? Use an appropriate statistical test or technique and support your choice.

An idea could be fitted a linear regression, where y=W vote score is the dependent variable and the time is the predictor.

In [ ]:
t1=df[ 'weighted_vote_score']
t1=np.array(t1)
t2=(df.timestamp_updated - df.timestamp_created)
t2=np.array(t2)
t2=t2.reshape((-1, 1))


In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg=LinearRegression()
lin_reg.fit(t2,t1)
ypredict=lin_reg.predict(t2)

plt.scatter(t2,t1)
plt.plot(t2,lin_reg.predict(t2),color='green')
plt.title("Regression Model")
plt.xlabel("Time pass")
plt.ylabel("Score")
#bella schifezza

In [ ]:
print('linear regression coefficient=', lin_reg.coef_[0])
print('linear regression intercept=', lin_reg.intercept_)

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
print ("Coefficient of determination :",r2_score(t1,ypredict))
print ("MSE: ",mean_squared_error(t1,ypredict))
print("RMSE: ",np.sqrt(mean_squared_error(t1,ypredict)))
#i commenti li farò meglio a dataset completo

The value 𝑏₀ = 0.14 (approximately) illustrates that the model predicts the response 0.14 when 𝑥 (time that pass) is zero. The value 𝑏₁ = 1.2e-09 means that the predicted response rises by 𝑏₁ when 𝑥 is increased by one (in this case: the time increases of one "what??.

### Is there any change in the relationship of the variables mentioned in the previous literal if you include whether an application is recommended or not in the review? Use an appropriate statistical test or technique and support your choice.

In [ ]:
import statsmodels.api as sm

In [ ]:
t3=df['recommended'].astype(int)


X=np.vstack([t1,t3])
X = X.transpose()
print(X.shape)
print(t1.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
l_reg=LinearRegression()

model=l_reg.fit(X,t1)
#print(model)

In [ ]:

# Make predictions
expected = t1
predicted = model.predict(X)


print('linear regression coefficient=', lin_reg.coef_[0])
print('linear regression intercept=', lin_reg.intercept_)
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
print ("Coefficient of determination :",r2_score(t1,predict))
print ("MSE: ",mean_squared_error(t1,predict))
print("RMSE: ",np.sqrt(mean_squared_error(t1,predict)))
#i commenti li farò meglio a dataset completo


### What are histograms, bar plots, scatterplots and pie charts used for?

A histogram is visula representation of the distribution of numerical data usually grouped in bins which a non 
overlapped series of intervals.
A bar chart or bar graph is a chart or graph that presents categorical data with rectangular bars with heights 
or lengths proportional to the values that they represent.
A scatter plot uses dots to represent values for two different numeric variables expressing the relationship that 
occurs between the two variables.
A pie chart is a circular graphic method which is used to illustrate numerical proportion into a group of variables.


### What insights can you extract from a Box Plot?

The box-plot is a graphical representation that can be used to describe the distribution of data through 5 indices: the minimum, the maximum, the median (quantile lev=0.5) and the first and third quartiles.
The "box" is delimited by the first and third quartiles (so the height is the interquantile difference IQR = q3-q1)and divided inside by the median. The "whiskers" are delimited by the minimum and maximum of the values. In this way the data is divided into four intervals with the same number of elements and this shows, for example, if the distribution is symmetric or not. Data that does not fit into the "whiskers" are called outliers, they are defined as outside the interval [-IQR,+IQR].

# Bonus points

In [ ]:
'''
For this homework, you are required to work with all data in the steam_reviews.csv. An extension (two files) 
of the dataset is available in the next links:

a. File 1 to be downloaded from https://sapienza2021adm.s3.eu-south-1.amazonaws.com/steam_reviews_bonus_1.zip.

b. File 2 to be downloaded from https://sapienza2021adm.s3.eu-south-1.amazonaws.com/steam_reviews_bonus_2.zip.

It is not necessary to use the extension for this homework, however, if you decide to use it, we will take it into
account in the final evaluation. In summary, to get the bonus points you are required to work with
[steam_reviews.csv + two files of extension] all together.
''';


# Theoretical Questions

# TQ1

In [ ]:
'''
We are given the following algorithm.

Input: 
    A: array of length n
    k: integers between 1 and n
    
function alg(A, k):
  s <-- a random element of A
  set L = [all the elements of A with value <= s]
  set R = [all the elements of A with value > s]
  r = len(L)
  if k == r:
    return s
  else if k < r:  
    return alg(L, k)
  else:
    return alg(R, k - r)
''';
import random
#IMPLEMENTATION
'''
def alg(A, k):
    #s= a random element of A
    s=random.choice(A)
    print(s)
    L = [x for x in A if x<=s]
    print('L=',L)
    R = [x for x in A if x>s]
    print('R=',R)
    lung = len(L)
    if k == lung:
        return s
    elif k < lung:  
        return alg(L, k)
    else:
        return alg(R, k - lung)
    
alg(A,k)
''';

### What does the algorithm compute?

The algorith find the k smallest element in the Array.

### What is asymptotically (i.e., we are asking for big-O complexity) the running time of the algorithm in the worst case, as a function of n?

In [ ]:
The worst case occurs when s=min(A)  and k=len(A).
In this situation: $$\begin{align}T(n) &= n + T(n-1)\\&= n + (n-1) + T(n-2)\\&= ...\\&=   \sum_{i=0}^{n}n-i\\&= (\sum_in + \sum_i i)\\&= ...\\&= n^2 - n\\&=O(n^2) \end{align}$$

There are degenerate cases where we can not guarantees the convergence of the algorithm:
$$ \begin{itemize}
    \ K=0
    \ s=max(A): the probability is almost 0, but we can not exclude this behaviour
    \ the element we are looking is repeated in the array
\end{itemize} $$

In [ ]:
### What is asymptotically the running time of the algorithm in the best case?

The best case occurs for k=len(L) and the running time is $$ T(n)=1+n+1+1= O(n) $$

The medium running time is $$ O(n) $$

# TQ2

In [ ]:
'''
You are given the recursive function splitSwap, which accepts an array a, an index i, and a length n.

function splitSwap(a, l, n):
  if n <= 1:
    return
  splitSwap(a, l, n/2)
  splitSwap(a, l+ n /2, n/2)
  swapList(a, l, n)
  
The subroutine swapList is described here:

function swapList(a, l, n):
  for i = 0 to n/2:
    tmp = a[l + i]
    a[l + i] = a[l + n/2 + i]
    a[l + n/2 + i] = tmp
'''
#IMPLEMENTATION


def splitSwap(a, l, n, lvl=0):
    if n <= 1:
        return
    indent = '\t'*lvl
    print(f"{indent}entered at lvl {lvl} with a: {a}\n")
    splitSwap(a, l, n//2, lvl+1)
    splitSwap(a, l+ n//2, n//2, lvl+1)
    print(f"{indent}* a pre_swap: {a}")
    swapList(a, l, n)
    print(f"{indent}* a after_swap: {a}\n\n")

def swapList(a, l, n):
    for i in range(n//2):
        tmp = a[l + i]
        a[l + i] = a[l + n//2 + i]
        a[l + n//2 + i] = tmp
    
splitSwap([i for i in range(1,9)], 0, 8)




### How much running time does it take to execute splitSwap(a, 0, n)? (We want a Big O analysis.)

In [ ]:
# Alessandro ha il codice latex

### What does this algorithm do? Is it optimal? Describe the mechanism of the algorithm in details, we do not want to know only its final result. HINT: Consider the scenario where len(a) and n are numbers that are a power of 2.

In [ ]:
The algorith reverse the subarray starting from the index i untill i+n : as the implementation shows, in every
level we swap a subarray of dimension 2^(log2(n)-1-level).
The algorith is not optimal since we can solve the same problem with an algorith with O(n)=n.
For example we can consider another array B in which we store the elements of the Array A from the index l+n untill l 
( step=-1) and then we could copy the elements of B in the positions l untill l+n of A


In [ ]:
def another_algorith(A,l,n):
    B=[]
    for i in range(l+n,l,-1):
        B.append(A[i])
    for i in range(l,l+n):
        A[i]=B[i-l]
        

# TQ3

In [ ]:
'''
In the knapsack problem we are given n objects and each object i has a weight w_i and a value v_i.
We are also given a weight budget W. The problem is to select a set of objects with total weight bounded by W 
that maximized the sum of their values. The following are three natural heuristics:

*Order them in increasing order of weight and then visit them sequentially, adding them to the solution 
as long as the budget is not exceeded

*Order them in decreasing order of values, and then visit them sequentially, adding them to the solution 
if the budget is not exceeded

*Order them in decreasing relative value (v_i / w_i), and then visit them sequentially, adding them to the 
solution if the budget is not exceeded
''';

### For each of the heuristics, provide a counterexample, that is, an example of a problem instance in which the heuristic fails to provide the optimal solution.

### First 

Suppose the existence of two elements A={p=2,v=2}, B={p=1,v=0} and set the weight budget=W=2.
If we order the objects in increasing order of weight and then visit them sequentially, adding them to the solution
as long as the budget is not exceeded , we will find {B} with a total value of 0.
This is not an optimal solution since the solution given by {A} respects the weight costraint 
and has a total value of 2.

### Second

Suppose the existence of three elements A={v=3,p=3},B={v=2,p=1},C={p=1,v=2} and set the weight budget=3.
If we order them in decreasing order of values, and then visit them sequentially, adding them to the 
solution if the budget is not exceeded, we will find {A} with a total value of 3.
This is not an optimal solution since the solution given by {B,C} has a total weight of 2 and a total value of 4.


### Third

Suppose the existence of two elements A={v=2,p=1,v/p=2}, B={v=3,p=3,v/p=1} and set the weight budget=3.
If we order them in decreasing relative value (v_i / w_i), and then visit them sequentially, adding them
to the solution if the budget is not exceeded, we will find {A} with a total value of 2.
This is not an optimal solution since the solution given by {B} respects the weight costraint 
and has a total value of 3.